# @1
# GHI CHÚ
# I) File này dùng để tổng hợp và phân loại dựa trên danh sách mặt hàng xuất nhập trong năm. Sau đó tạo và đưa dữ liệu vào bảng MAT_HANG trong CSDL Ban_Hang
# II) Sử dụng dữ liệu từ sổ xuất nhập tồn

In [6]:
import pandas as pd
import numpy as np
import os
import pyodbc
import pandas as pd

# B1 Tổng hợp 12 file xuất nhập tồn theo tháng của mỗi năm

In [7]:
def Process_Product(directory,skip_rows=4):
    combined_df = pd.DataFrame()
    for i in range(1, 13):
        # Construct the file name
        file_name = os.path.join(directory, f"TONG_HOP_TON_KHO {i}.xlsx")

        # Read the Excel file into a DataFrame
        df = pd.read_excel(file_name,skiprows = skip_rows)

        # Remove the last record from the DataFrame
        df = df[:-1]

        # Select columns from index 1 to 3
        df_selected = df.iloc[:, 1:4]

        # Append the selected columns DataFrame to the combined DataFrame
        combined_df = combined_df.append(df_selected, ignore_index=True)
        
    # Drop duplicates to keep only distinct values
    combined_df = combined_df.drop_duplicates()
    #Reset index
    combined_df = combined_df.reset_index(drop=True)
    # Rename column
    combined_df = combined_df.rename(columns={
                       'Unnamed: 1': 'Mã hàng',
                       'Unnamed: 2': 'Tên hàng',
                       'Unnamed: 3': 'ĐVT'}
                                )
    return combined_df

In [8]:
# Directory where Excel files are located
nam_2020 = 'D://THUCTAP//XUAT NHAP TON 2020//'
nam_2021 = 'D://THUCTAP//XUAT NHAP TON 2021//'
nam_2022 = 'D://THUCTAP//XUAT NHAP TON 2022//'

In [9]:
DSSP_20 = Process_Product(nam_2020)
DSSP_21 = Process_Product(nam_2021)
DSSP_22 = Process_Product(nam_2022,3)

In [10]:
DSSP_22

,Mã hàng,Tên hàng,ĐVT
0,BH_150G,Bò hầm 150g,Thùng
1,BH_150GL,Bò hầm (lon in) hộp 150g,Hộp
2,BV_BO500G,Bò viên 3 ngon_gói 500g,Gói
3,BX_170G,Bò xay 170g,Thùng
4,CA_FUNNY500,Cá viên Funny gói 500g,Gói
...,...,...,...
215,TL_MEVIOSOB,Thuốc lá điếu lọc Mevius Original Blue,Gói
216,NY_HC460KM,Nước yến hạt chia Nutizen chai 460ml,Chai
217,TL_LOTUSRCND,Thuốc lá bao Lotus nâu - RCNĐ,Bao
218,X_BB20GH1KG,XXTT Boom Boom 20g hũ 1kg,Hũ


# B2 Phân loại hàng dựa trên kí tự trong mã hàng

In [11]:
# Lấy ký tự từ Mã hàng để phân loại hàng
def extract(value):
    return value.split('_')[0]

In [12]:
# phân loại hàng
def map_chr_to_new_column(chr_value):
    if chr_value == 'TL':
        return 'Thuốc lá'
    elif chr_value in ['X','XX','CHA','BH','HH','BV','C','H','PT','HN','BX','G','HQ']:
        return 'SP từ Thịt'
    else:
        return 'Các SP khác'

In [13]:
def Apply(df):
    df['chr'] = df['Mã hàng'].apply(extract)
    df['Loại hàng'] = df['chr'].apply(map_chr_to_new_column)
    df.drop(columns=['chr'], inplace=True)
    return df

In [14]:
DSSP20 = Apply(DSSP_20)
DSSP21 = Apply(DSSP_21)
DSSP22 = Apply(DSSP_22)

In [15]:
DSSP21

,Mã hàng,Tên hàng,ĐVT,Loại hàng
0,BH_150G,Bò hầm 150g,Thùng,SP từ Thịt
1,BH_150GM,Bò hầm 150g,Hộp,SP từ Thịt
2,BV_BO500G,Bò viên 3 ngon_gói 500g,Gói,SP từ Thịt
3,BX_170G,Bò xay 170g,Thùng,SP từ Thịt
4,C_CNXD,Cá ngừ xốt dầu,Thùng,SP từ Thịt
...,...,...,...,...
156,CF_SNGV240G,Ông Bầu OB cà phê sữa nóng hoà tan hộp giấy và...,Hộp,Các SP khác
157,CF_TGV240G,OB cà phê trứng giấy vàng 240g (10 gói x 24g. ...,Hộp,Các SP khác
158,HH_150GL,Heo hầm (lon in) hộp 150g,Hộp,SP từ Thịt
159,X_BRMO500,Ba rọi xốt mật ong_gói 500g,Gói,SP từ Thịt


In [16]:
DSSP20.to_csv('D://THUCTAP//data_da_xuly//DSSP20.csv', index=False, encoding='utf-16', sep='\t')
DSSP21.to_csv('D://THUCTAP//data_da_xuly//DSSP21.csv', index=False, encoding='utf-16', sep='\t')
DSSP22.to_csv('D://THUCTAP//data_da_xuly//DSSP22.csv', index=False, encoding='utf-16', sep='\t')


# B3 ĐƯA DỮ LIỆU VÀO CSDL

In [17]:
# Kết nối với CSDL
conn = pyodbc.connect(
Trusted_Connection = "Yes",
Driver = '{ODBC Driver 17 for SQL Server}',
Server = "DESKTOP-MDDIIDJ\MSSQLSERVER01",
Database = 'Ban_Hang')
cursor = conn.cursor()

In [18]:
#Tạo bảng MAT_HANG
#cursor.execute("CREATE TABLE MAT_HANG(Ma_hang varchar(20) PRIMARY KEY,Ten_hang nvarchar(150),DVT nvarchar(10),Loai_hang nvarchar(15))")

In [19]:
#Tạo hàm để đẩy dữ liệu vào bảng MAT_HANG trong CSDL
def Insert_Mat_hang(df):
    for row in df.itertuples():
        cursor.execute('''
                        IF NOT EXISTS (SELECT 1 FROM BAN_HANG.dbo.MAT_HANG WHERE Ma_hang = ?)
                        BEGIN
                            INSERT INTO BAN_HANG.dbo.MAT_HANG(Ma_hang, Ten_hang, DVT, Loai_hang)
                            VALUES(?, ?, ?, ?)
                        END
                        ''',
                        row[1],
                        row[1],  # Ma_hang
                        row[2],  # Ten_hang
                        row[3],  # DVT
                        row[4]   # Loai_hang
                      )
    conn.commit()

In [20]:
#Insert_Mat_hang(DSSP20)
#Insert_Mat_hang(DSSP21)
#Insert_Mat_hang(DSSP22)